In [ ]:
%pip install openai numpy matplotlib pydantic --upgrade --quiet

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field
import os
from openai import OpenAI
import openai
from typing import Union

In [ ]:
os.environ["OPENAI_API_KEY"] = "set your API key here"
MODEL = "gpt-4.1-mini"

In [6]:
class KeyPoints(BaseModel):
    key_points: list[str] = Field(description="A list of key points extracted from the text.")

class KeyPointsAndSummary(KeyPoints):
    summary: str = Field(description="A summary of this text.")

client = OpenAI()

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "user", "content": "James is awesome, and likes to learn code and new things!"}
    ],
    response_format=KeyPointsAndSummary
)

print(completion)

ParsedChatCompletion[KeyPointsAndSummary](id='chatcmpl-ADDmMRySmjQ0wltxarGQYpBc0CvMR', choices=[ParsedChoice[KeyPointsAndSummary](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[KeyPointsAndSummary](content='{"key_points":["James is a positive person.","He enjoys learning about coding.","James is interested in acquiring new skills."],"summary":"James is an enthusiastic individual who has a passion for learning, particularly in the field of coding and other new subjects."}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=KeyPointsAndSummary(key_points=['James is a positive person.', 'He enjoys learning about coding.', 'James is interested in acquiring new skills.'], summary='James is an enthusiastic individual who has a passion for learning, particularly in the field of coding and other new subjects.')))], created=1727714594, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f8

In [7]:
key_points = KeyPointsAndSummary.model_validate_json(completion.choices[0].message.content)
print(key_points)

key_points=['James is a positive person.', 'He enjoys learning about coding.', 'James is interested in acquiring new skills.'] summary='James is an enthusiastic individual who has a passion for learning, particularly in the field of coding and other new subjects.'


In [10]:
class Table(str, Enum):
    fruits = "fruits"

class Column(str, Enum):
    id = "id"
    color = "color"
    ripeness = "ripeness"
    picked_at = "picked_at"
    sold_at = "sold_at"
    stored_at = "stored_at"
    discarded_at = "discarded_at"

In [12]:
class Operator(str, Enum):
    eq = "eq"
    gt = ">"
    lt = "<"
    le = "<="
    ge = ">="
    ne = "!="

In [14]:
class OrderBy(str, Enum):
    asc = "asc"
    desc = "desc"

class DynamicValue(BaseModel):
    column_name: str

class Condition(BaseModel):
    column: str
    operator: Operator
    value: Union[str, int, DynamicValue]

class Query(BaseModel):
    table_name: Table
    columns: list[Column]
    conditions: list[Condition]
    order_by: OrderBy

In [24]:
completion = client.chat.completions.create(model=MODEL, messages=[
    {'role': 'system', 'content': "You are a helpful assistant that extracts information from a user's request."},
    {'role': 'user', 'content': "I want to query the 'fruits' table to get all the apples that are ripe and sold after 2024-01-01."}
], tools=[openai.pydantic_function_tool(Query)])


In [31]:
import json
json.loads(completion.choices[0].message.tool_calls[0].function.arguments) 

{'table_name': 'fruits',
 'columns': ['id',
  'color',
  'ripeness',
  'picked_at',
  'sold_at',
  'stored_at',
  'discarded_at'],
 'conditions': [{'column': 'type', 'operator': 'eq', 'value': 'apple'},
  {'column': 'ripeness', 'operator': 'eq', 'value': 'ripe'},
  {'column': 'sold_at', 'operator': '>', 'value': '2024-01-01'}],
 'order_by': 'asc'}

In [33]:
completion = client.chat.completions.create(model=MODEL, messages=[
    {'role': 'system', 'content': "You are a helpful assistant that extracts information from a user's request."},
    {'role': 'user', 'content': "Hi how are you?"}
], tools=[openai.pydantic_function_tool(Query)])

print(completion.choices[0].message.content)

I'm doing great, thank you! How can I assist you today?
